In [1]:
import pandas as pd
import numpy as np

import json
import re

In [2]:
BASE_URL = 'https://docs.google.com/spreadsheets/d/'

In [3]:
sheet_url = '1zJ_3AarYrP4MNqM4yAlgyisMZ9z2_MBL6hZdwj-mmYg/export?format=xlsx'

raw_df = pd.read_excel(BASE_URL+sheet_url, sheet_name='responses')
raw_df

,date,id,answers
0,"2/2/2025, 3:25:41 PM",bnluh0zrwid,"[{""question"":""Pregunta 1"",""confidence"":""0"",""an..."
1,"2/2/2025, 3:26:38 PM",bnluh0zrwid,"[{""question"":""Pregunta 1"",""confidence"":""0"",""an..."
2,"2/2/2025, 3:28:37 PM",bnluh0zrwid,"[{""question"":""Pregunta 1"",""confidence"":""0"",""an..."
3,"2/2/2025, 4:10:11 PM",bnluh0zrwid,"[{""question"":""Pregunta 1"",""confidence"":""0"",""an..."
4,"2/2/2025, 4:14:16 PM",bnluh0zrwid,"[{""question"":""Pregunta 1"",""confidence"":""0"",""an..."
5,"2/2/2025, 4:17:58 PM",bnluh0zrwid,"[{""question"":""Pregunta 1"",""confidence"":""0"",""an..."
6,"2/2/2025, 4:18:11 PM",bnluh0zrwid,"[{""question"":""Pregunta 1"",""confidence"":""0"",""an..."
7,"2/2/2025, 4:41:35 PM",bnluh0zrwid,"[{""question"":""Pregunta 1"",""confidence"":""0"",""an..."


In [4]:
raw_df.date = pd.to_datetime(raw_df.date, format='%d/%m/%Y, %I:%M:%S %p')

df = list()
for date, idx, answers in raw_df.values:
    answers = json.loads(answers)
    for answer in answers:
        row = {
            'date': date, 'idx': idx, 'question': answer['question'],
            'confidence': answer['confidence'], 'answer': answer['answer']
        }
        for option in answer['options']:
            label = option['label'][0].lower()
            row.update({f'{label}_{option_key}': option_value for option_key, option_value in option.items()})
        df.append(row)

df = pd.DataFrame(df)
df.question = df.question.str.replace(r'\D+', '', regex=True).astype(np.uint8)
df.confidence = df.confidence.astype(np.uint8)
df.answer = df.answer.str.lower().astype('category')
cols_order = ['idx', 'date', 'question', 'answer', 'confidence']
for prefix in 'abcde':
    if prefix+'_label' not in df.columns:
        continue
    df[f'{prefix}_label'] = df[f'{prefix}_label'].str[0].str.lower().astype('category')
    df[f'{prefix}_discarded'] = df[f'{prefix}_discarded'].astype(np.uint8)
    df[f'{prefix}_stars'] = df[f'{prefix}_highlightedStars'].astype(np.uint8)
    df.drop(columns=f'{prefix}_highlightedStars', inplace=True)
    cols_order.extend([f'{prefix}_label', f'{prefix}_discarded', f'{prefix}_stars'])
df = df[cols_order]
df.sort_values(['date', 'idx'], inplace=True)
df.drop_duplicates(['idx', 'question'], keep='last', inplace=True)
df.reset_index(drop=True, inplace=True)
df

,idx,date,question,answer,confidence,a_label,a_discarded,a_stars,b_label,b_discarded,b_stars,c_label,c_discarded,c_stars,d_label,d_discarded,d_stars
0,bnluh0zrwid,2025-02-02 16:41:35,1,c,0,a,1,5,b,0,0,c,0,0,d,0,0
1,bnluh0zrwid,2025-02-02 16:41:35,2,b,0,a,0,0,b,0,0,c,0,0,d,0,0
2,bnluh0zrwid,2025-02-02 16:41:35,3,a,0,a,0,1,b,0,0,c,0,0,d,0,0
3,bnluh0zrwid,2025-02-02 16:41:35,4,c,0,a,0,0,b,0,0,c,0,0,d,0,0
4,bnluh0zrwid,2025-02-02 16:41:35,5,d,0,a,0,0,b,0,0,c,0,0,d,0,0
5,bnluh0zrwid,2025-02-02 16:41:35,6,a,0,a,0,0,b,0,0,c,0,0,d,0,0
6,bnluh0zrwid,2025-02-02 16:41:35,7,d,0,a,0,0,b,0,0,c,0,0,d,0,0
7,bnluh0zrwid,2025-02-02 16:41:35,8,b,0,a,0,0,b,0,0,c,0,0,d,0,0
8,bnluh0zrwid,2025-02-02 16:41:35,9,a,0,a,0,0,b,0,0,c,0,0,d,0,0
9,bnluh0zrwid,2025-02-02 16:41:35,10,d,0,a,0,0,b,0,0,c,0,0,d,0,0
